In [7]:
import builder
import ItemCBFKNNRecommender as calc_sim
import evaluation_function
from evaluation_function import evaluate_algorithm
from hybridRecommender import HybridRecommender


In [8]:
b=builder.Builder()

In [5]:
#b.get_train_pd().to_csv('new_train.csv')
b.get_train_pd()[b.get_train_pd()['playlist_id']==50431]['track_id']

1211782     1830
1211783    11232
1211784     8539
1211785    17818
1211786    20371
1211787     3535
1211788     4526
1211789    18547
1211790    19091
Name: track_id, dtype: int64

In [9]:
import imp
imp.reload(builder)
imp.reload(calc_sim)
imp.reload(evaluation_function)
#imp.reload(hybridRecommender)

<module 'evaluation_function' from '/home/strenuus/dev/recommender-systems/alternative-hybrid/evaluation_function.py'>

In [12]:
#b = builder.Builder()
URM_train, URM_test = b.train_test_holdout(1)

In [13]:
 contentSimilarity, collaborativeSimilarity, userbasedSimilarity = compute_similarity()

Similarity column 20600 ( 100 % ), 2941.01 column/sec, elapsed time 0.12 min
Similarity column 16800 ( 81 % ), 558.95 column/sec, elapsed time 0.50 min
Similarity column 20600 ( 100 % ), 557.56 column/sec, elapsed time 0.62 min
Similarity column 16900 ( 34 % ), 562.57 column/sec, elapsed time 0.50 min
Similarity column 34200 ( 68 % ), 566.78 column/sec, elapsed time 1.01 min
Similarity column 50400 ( 100 % ), 577.63 column/sec, elapsed time 1.45 min


In [11]:
def compute_similarity():
    recommender = calc_sim.ItemCBFKNNRecommender(URM_train, b.get_ICM(0.7))#artist value
    contentSimilarity = recommender.fit(shrink=0.0, topK=200)
    recommender = calc_sim.ItemCBFKNNRecommender(URM_train, b.get_URM_transpose())
    collaborativeSimilarity = recommender.fit(shrink=5.0, topK=500)
    recommender = calc_sim.ItemCBFKNNRecommender(URM_train, b.get_URM())
    userbasedSimilarity = recommender.fit(shrink=5.0, topK=500)
    return contentSimilarity, collaborativeSimilarity, userbasedSimilarity

def run_local():
    return evaluate_algorithm(URM_test, HybridRecommender(contentSimilarity, collaborativeSimilarity, userbasedSimilarity, 0.1, 1),b)#content value,collaborative value

def run_online():
    print_to_csv(contentSimilarity, collaborativeSimilarity, userbasedSimilarity)

def print_to_csv(contentSimilarity, collaborativeSimilarity, userbasedSimilarity):
    file=open("hybrid-submission.csv",'a')
    file.write("playlist_id,track_ids"+"\n")
    recommender = HybridRecommender(contentSimilarity, collaborativeSimilarity, userbasedSimilarity, 0.1, 1)
    for playlist in b.get_ordered_target_playlists():
        s = str(recommender.recommend(playlist,b))
        s = s[1:len(s)-1]
        file.write(str(playlist)+","+s+"\n")
    for playlist in b.get_unordered_target_playlists():
        s = str(recommender.recommend(playlist,b))
        s = s[1:len(s)-1]
        file.write(str(playlist)+","+s+"\n")

In [ ]:
run_online()

[[-0.040161047130823135 6667]
 [-0.06853228062391281 9223]
 [-0.09544509649276733 14962]
 [-0.1428571194410324 442]
 [-0.06853228062391281 2404]
 [-0.05591559037566185 5109]
 [-0.10557280480861664 8473]
 [-0.06417246907949448 12111]
 [-0.15590372681617737 12424]
 [-0.0859604999423027 12568]
 [0 16190]]


Similarity column 20600 ( 100 % ), 2687.33 column/sec, elapsed time 0.13 min
Similarity column 17100 ( 83 % ), 567.74 column/sec, elapsed time 0.50 min
Similarity column 20600 ( 100 % ), 571.47 column/sec, elapsed time 0.60 min
[[-3.0806736120810894 5907]
 [-3.0806736120810894 9405]
 [-3.5497164772107057 15328]
 [-3.0806736120810894 19158]
 [-0.3001874456038842 1831]
 [-0.7692303107335019 2173]
 [-0.7692303107335019 4811]
 [-0.3001874456038842 11989]
 [-0.5403374020869919 9130]
 [-0.9155716941906861 9365]
 [0 13632]]
[[-0.39024367928504944 1089]
 [-0.22299638816288547 256]
 [-0.22299638816288547 15814]
 [-0.22299638816288547 18644]
 [-0.11149819408144278 3204]
 [-0.11149819408144278 9961]
 [-0.11149819408144278 10841]
 [-0.11149819408144278 14742]
 [-0.11149819408144278 16662]
 [0 13859]
 [0 10022]]
[[-4.118464495454518 1588]
 [-3.290938869118692 4850]
 [-2.8989530461175113 6426]
 [-3.7700326527868007 8540]
 [-4.118464495454518 12668]
 [-4.118464495454518 13402]
 [-3.290938869118692 13

KeyboardInterrupt: 